# Compiling lattice surgery schedules

Requires configuring gridsynth and LatticeSurgerySimulator.

In [ ]:
import os
try:
    path_initialized
except NameError:
    path_initialized = True
    os.chdir('..')

from benchmarks.cirq_to_ls import cirq_to_ls
from swiper.simulator import DecodingSimulator
from swiper.lattice_surgery_schedule import LatticeSurgerySchedule
import matplotlib.pyplot as plt
import swiper.plot as plotter
import subprocess
from benchmarks.benchmark_circuits import QROM, CarlemanEncoding, ElectronicStructure, FermiHubbardEncoding, HeisenbergEncoding, ChemicalHamiltonianEncoding, RegularT, RandomT, Memory, MSD15To1, QASMBenchmark, Toffoli, RZ
from pyLIQTR.clam.lattice_definitions import KagomeLattice, TriangularLattice
import pickle as pkl

In [ ]:
# Specify which benchmarks to generate
benchmark_param_combos = [
    # (Memory, (10000,)),
    # (MSD15To1, ()),  
    # (Toffoli, ()),
    # (RegularT, (1000, 0)),
    # (RandomT, (1000, 200, 0)),
    # (RandomT, (10000, 200, 0)),
    # (QROM, (None, 15, 15)),
    # (CarlemanEncoding, (2, 4)),
    # (ElectronicStructure, ()),
    #(FermiHubbardEncoding, ((2,2),)),
    #(FermiHubbardEncoding, ((4,4),)),
    #(FermiHubbardEncoding, ((2,2,3), KagomeLattice, 'Kagome')),
    (FermiHubbardEncoding, ((2,3,3), KagomeLattice, 'Kagome')),
    # (HeisenbergEncoding, (3,)),
    # (HeisenbergEncoding, (5,)),
    # #(HeisenbergEncoding, (10,)),
    # #(ChemicalHamiltonianEncoding, (112,)), # H2O
    # (ChemicalHamiltonianEncoding, (140,)), # H2
    #(ChemicalHamiltonianEncoding, (146,)),  # LiH
]

In [ ]:
# RandomT schedules
for i in range(20):
    b = RandomT(1000, 200, i)
    schedule = b.get_schedule()
    name = b.name()
    savepath = f'benchmarks/cached_schedules/random_t/{name}.lss'
    with open(savepath, 'w') as f:
        f.write(str(schedule))

In [ ]:
# Rz Schedules
for prec in [1e-5, 1e-10, 1e-15, 1e-20]:
    b = RZ(prec, 1)
    schedule = b.get_schedule()
    name = b.name()
    savepath = f'benchmarks/cached_schedules/{name}.lss'
    with open(savepath, 'w') as f:
        f.write(str(schedule))
for seq in [10, 100]:
    b = RZ(1e-10, seq)
    schedule = b.get_schedule()
    name = b.name()
    savepath = f'benchmarks/cached_schedules/{name}.lss'
    with open(savepath, 'w') as f:
        f.write(str(schedule))

In [ ]:
for benchmark, params in benchmark_param_combos:
    print(f'Generating benchmark {benchmark.__name__} with params {params}')
    b = benchmark(*params)
    schedule = b.get_schedule()
    print(f'\tInstruction count: {len(schedule)}')
    name = b.name()
    savepath = f'benchmarks/cached_schedules/{name}.lss'
    print(f'\tSaving to {savepath}...')
    with open(savepath, 'w') as f:
        f.write(str(schedule))
    print(f'\tFile size: {os.path.getsize(savepath) / 2**20: 0.2f} MB')

In [ ]:
# MQT benchmarks (QPE and Shor)

for file in os.listdir('benchmarks/data/qasm/'):
    print(file, end='...')
    b = QASMBenchmark(f'benchmarks/data/qasm/{file}')
    schedule = b.get_schedule()
    print(f'\tInstruction count: {len(schedule)}')
    name = b.name()
    savepath = f'benchmarks/cached_schedules/{name}.lss'
    print(f'\tSaving to {savepath}...')
    with open(savepath, 'w') as f:
        f.write(str(schedule))
    print(f'\tFile size: {os.path.getsize(savepath) / 2**20: 0.2f} MB')

In [ ]:
ls_schedule = LatticeSurgerySchedule.from_str(open('benchmarks/cached_schedules/toffoli.lss').read())

In [ ]:
d=7
decoding_time = 0
speculation_time = 0
speculation_accuracy = 1

simulator = DecodingSimulator()

In [ ]:
#%%prun
success, sim_params, device_data, window_data, decoding_data = simulator.run(
    schedule=ls_schedule,
    distance=d,
    decoding_latency_fn=lambda _: decoding_time,
    speculation_mode='integrated',
    scheduling_method='sliding',
    max_parallel_processes=None,
    progress_bar=True,
    save_animation_frames=False
)

In [ ]:
plotter.plot_device_schedule_trace(device_data, hide_z_ticks=True)